In [20]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [21]:

planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [22]:
#### real web search : use mocked function below to understand  better 
from dotenv import load_dotenv
load_dotenv()
from langchain_community.utilities import GoogleSerperAPIWrapper

from autogen_ext.tools.http import HttpTool


SERPER_API_KEY=os.getenv("SERPER_API_KEY")


search_tool_wrapper = GoogleSerperAPIWrapper(type='search',serper_api_key =SERPER_API_KEY)

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."  

In [23]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [24]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web], ## pass search_web_tool for mock
    reflect_on_tool_use=False,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [25]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

In [26]:
data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

#### Termination condition

In [27]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

txt_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = txt_mention_termination | max_message_termination


In [28]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [29]:
selector_team = SelectorGroupChat(
    participants=[planning_agent,web_search_agent,data_analyst_agent],
    model_client=model_client,
    termination_condition= combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True

)

In [30]:

task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [31]:
from autogen_agentchat.ui import Console
await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To find the information requested, we need to follow these steps:

1. WebSearchAgent: Search for the Miami Heat player with the highest points during the 2006-2007 NBA season.
2. WebSearchAgent: Search for this player's total rebounds in the 2007-2008 season.
3. WebSearchAgent: Search for this player's total rebounds in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds between the 2007-2008 season and the 2008-2009 season.

Let's start with these steps.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_z6MGdhfHPHk3bsagXH1Gwt1J', arguments='{"query":"Miami Heat highest points player 2006-2007 season"}', name='search_web')]
---------- ToolCallExec

TaskResult(messages=[TextMessage(id='30d9e6c0-411b-4739-a724-401835924b88', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 47, 8, 597366, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='766335e5-1a60-4ec5-a77a-e434789bbb78', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=123), metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 47, 11, 844087, tzinfo=datetime.timezone.utc), content="To find the information requested, we need to follow these steps:\n\n1. WebSearchAgent: Search for the Miami Heat player with the highest points during the 2006-2007 NBA season.\n2. WebSearchAgent: Search for this player's total rebounds in the 2007-2008 season.\n3. WebSearchAgent: Search for this player's tota

In [32]:

state = await selector_team.save_state()

In [33]:
state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'PlanningAgent': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': "To find the information requested, we need to follow these steps:\n\n1. WebSearchAgent: Search for the Miami Heat player with the highest points during the 2006-2007 NBA season.\n2. WebSearchAgent: Search for this player's total rebounds in the 2007-2008 season.\n3. WebSearchAgent: Search for this player's total rebounds in the 2008-2009 season.\n4. DataAnalystAgent: Calculate the percentage change in total rebounds between the 2007-2008 season and the 2008-2009 season.\n\nLet's st

### custom selector function:

In [36]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source == web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team1 = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)

In [37]:
# With real web search
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.ui import Console
await Console(selector_team1.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To provide you with the information requested, we’ll need to complete the following tasks:

1. WebSearchAgent: Look up the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds this player had in the 2007-2008 season.
3. WebSearchAgent: Find the total rebounds this player had in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in the player’s total rebounds from the 2007-2008 season to the 2008-2009 season.

Let's proceed with these tasks.
---------- TextMessage (WebSearchAgent) ----------
Dwyane Wade was the Miami Heat player with the highest points during the 2006-2007 season.

- In the 2007-2008 season, Dwyane Wade averaged 4.2 rebo

TaskResult(messages=[TextMessage(id='34e1a15b-7adb-494b-b67f-ac2282611750', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 48, 23, 97120, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='cc3c11ea-2890-46d2-aa07-815e371b6c84', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=2809, completion_tokens=132), metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 48, 25, 502657, tzinfo=datetime.timezone.utc), content="To provide you with the information requested, we’ll need to complete the following tasks:\n\n1. WebSearchAgent: Look up the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. WebSearchAgent: Find the total rebounds this player had in the 2007-2008 season.\n3. WebSearchAgent: Find the t

In [38]:
await selector_team.reset()

In [39]:
# If Data analyst agent is not selected after web search agent
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False,
    selector_func=my_selector_fun)


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player's points during the 2006-2007 season and his total rebounds for the 2007-2008 and 2008-2009 seasons so I can perform the analysis.
---------- TextMessage (DataAnalystAgent) ----------
It seems I can provide some data on top scorers for the Miami Heat during the 2006-2007 season to help answer your question.

The Miami Heat player with the highest point per game in the 2006-2007 NBA season was Dwyane Wade, who averaged 27.4 points per game.

Please provide Dwyane Wade's total rebounds for the 2007-2008 and 2008-2009 seasons so I can calculate the percentage change in his total rebounds between those seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please p

TaskResult(messages=[TextMessage(id='5681d78c-5d95-414f-9c4d-bca1c9cbfd3c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 51, 38, 540260, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='ed62f66e-7874-4cb9-a014-2742cb2418f3', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=47), metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 51, 40, 456495, tzinfo=datetime.timezone.utc), content="Please provide the data for the Miami Heat player's points during the 2006-2007 season and his total rebounds for the 2007-2008 and 2008-2009 seasons so I can perform the analysis.", type='TextMessage'), TextMessage(id='f7a33d2e-d73d-4f01-9b17-43772d3543c5', source='DataAnalystAgent', models_usage=RequestUsa

### selector candidate function

In [41]:
from typing import List

In [42]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]

In [44]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    

    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    

    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
        

    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
# await sele.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To accomplish this, let's break it down into tasks:

1. **WebSearchAgent**: Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. **WebSearchAgent**: Find the total rebounds for that player in the 2007-2008 season.
3. **WebSearchAgent**: Find the total rebounds for that player in the 2008-2009 season.
4. **DataAnalystAgent**: Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (UserProxyAgent) ----------
again
---------- TextMessage (PlanningAgent) ----------
1. **WebSearchAgent**: Identify the Miami Heat player with the highest points in the 2006-2007 season.
---------- TextMessage (UserProxyAgent) ----------
aprrove
--------

TaskResult(messages=[TextMessage(id='8cf36262-532b-4367-9f58-f38452bf8bfc', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 56, 6, 414042, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='232287dd-bb95-4e91-9f8d-d2849b073b95', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=696, completion_tokens=121), metadata={}, created_at=datetime.datetime(2025, 9, 15, 8, 56, 7, 917049, tzinfo=datetime.timezone.utc), content="To accomplish this, let's break it down into tasks:\n\n1. **WebSearchAgent**: Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. **WebSearchAgent**: Find the total rebounds for that player in the 2007-2008 season.\n3. **WebSearchAgent**: Find the total rebounds for that player in